In [0]:
# Check if the mount point exists
mount_path = "/mnt/vision-test"
dbutils.fs.ls(mount_path)

In [0]:
# Unmount the directory
# dbutils.fs.unmount(f"/mnt/{mount_name}")

# Process Images

In [0]:
# Convert the list of FileInfo objects to a spark DataFrame

def create_file_into_df(source_dir:str):

    files = dbutils.fs.ls(f"{mount_path}/{source_dir}")

    file_info_list = [
        {
            "path": file.path,
            "name": file.name
        }
        for file in files
        ]

    return spark.createDataFrame(file_info_list)

file_info_df = create_file_into_df("images")
display(file_info_df)


In [0]:
label_files = dbutils.fs.ls(f"{mount_path}/labels")
label_path = label_files[0].path

label_df = spark.read.csv(label_path, header=True, inferSchema=True)
display(label_df)